In [2]:
import os
import sys
import logging

import pandas as pd
import numpy as np
import pysam
import shutil
from samtools_utils import *

BASEDIR = "/data1/shahs3/users/sunge/cnv_simulator"
profile_name = "minitest_c2_3"

BAMDIR = f"{BASEDIR}/synthetic_bams_2/{profile_name}"

In [4]:
cnv_profile_df = pd.read_csv(f"{BAMDIR}/{profile_name}_cell_profile.tsv",
                             sep="\t", header=0)
data_cnv_profile_df = cnv_profile_df.loc[cnv_profile_df["chr"] != 0]
clone_cnv_profile_df = cnv_profile_df.loc[cnv_profile_df["chr"] == 0]
clone_cell_count_lst = [
    len(row["cell_barcode"].split(","))
    for _, row in clone_cnv_profile_df.iloc[1:].iterrows()
]
print(clone_cnv_profile_df)

clone_cell_id_dict = {}
for _, clone in enumerate(clone_cnv_profile_df["clone"].unique()):
    if clone != -1:
        clone_cell_id_dict[f"clone{clone}"] = set(
            clone_cnv_profile_df.loc[clone_cnv_profile_df["clone"] == clone, "cell_barcode"].values[0].split(",")
        )
print(clone_cell_id_dict.keys())

   clone  chr  start  end  copy_number  state  size  \
0     -1    0      0    0            2      0   0.0   
1      0    0      0    0            2      0   0.0   
2      1    0      0    0            2      0   0.0   

                                        cell_barcode  \
0  128729A-R20-C19,A108762A-R08-C13,A108762B-R52-...   
1  128683A-R27-C25,A96123A-R26-C64,128663A-R49-C2...   
2  110720-R25-C49,A108761B-R54-C36,128723A-R29-C4...   

                                        sample_group  
0  8,7,11,11,8,7,6,3,1,1,8,10,11,11,3,0,1,1,1,11,...  
1  2,4,5,1,4,1,3,11,12,12,4,4,5,12,1,1,7,2,4,4,8,...  
2  11,9,4,11,6,10,3,5,8,7,1,2,5,11,10,2,12,9,9,0,...  
dict_keys(['clone0', 'clone1'])


In [5]:
baseline_bam_path = f"{BAMDIR}/{profile_name}_baseline_cells.bam"
synthetic_bam_path = f"{BAMDIR}/{profile_name}_final_sorted_cnv.bam"
del_region = "1:91936118-95477392"

baseline_region_counts = samtools_get_indexed_read_count(baseline_bam_path,
                                                         del_region,
                                                         cell_barcodes = clone_cell_id_dict["clone0"])
print(f"Baseline read count in {del_region}: {baseline_region_counts}")

synthetic_region_counts = samtools_get_indexed_read_count(synthetic_bam_path,
                                                         del_region,
                                                         cell_barcodes = clone_cell_id_dict["clone0"])
print(f"Synthetic read count in {del_region}: {synthetic_region_counts}")


FileNotFoundError: [Errno 2] No such file or directory: 'samtools'